In [2]:
import os
import socket
from pyspark.sql import SparkSession
from dotenv import load_dotenv # <--- Nova Importação

# 1. Forçar leitura do arquivo .env
# Isso garante que as variáveis existam mesmo se o shell falhar
load_dotenv("/home/andre/lakehouse_project/.env") 

# 2. Carregar variáveis (Agora com valor garantido ou padrão)
spark_master_host = os.getenv('SPARK_MASTER_HOST', '192.168.15.9') # Fallback para IP fixo se falhar
spark_master_port = os.getenv('SPARK_MASTER_PORT', '7077')         # Fallback para 7077
spark_master = f"spark://{spark_master_host}:{spark_master_port}"

minio_endpoint = os.getenv('MINIO_ENDPOINT')
minio_access = os.getenv('MINIO_ACCESS_KEY')
minio_secret = os.getenv('MINIO_SECRET_KEY')
hive_jdbc = os.getenv('HIVE_METASTORE_JDBC')
hive_user = os.getenv('HIVE_USER')
hive_pass = os.getenv('HIVE_PASSWORD')
worker_python = os.getenv('WORKER_PYTHON_PATH')
driver_python = os.getenv('DRIVER_PYTHON_PATH')

print(f"🔌 Conectando em: {spark_master}")
print(f"📦 Storage: {minio_endpoint}")

# Validação Rápida
if str(spark_master_port) == "None":
    raise ValueError("❌ ERRO CRÍTICO: Não foi possível ler o arquivo .env! Verifique se ele existe.")

spark = SparkSession.builder \
    .appName("Checkup_DevOps_Secure") \
    .master(spark_master) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.warehouse.dir", "s3a://warehouse/") \
    .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", minio_access) \
    .config("spark.hadoop.fs.s3a.secret.key", minio_secret) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.javax.jdo.option.ConnectionURL", hive_jdbc) \
    .config("spark.hadoop.javax.jdo.option.ConnectionDriverName", "org.postgresql.Driver") \
    .config("spark.hadoop.javax.jdo.option.ConnectionUserName", hive_user) \
    .config("spark.hadoop.javax.jdo.option.ConnectionPassword", hive_pass) \
    .config("spark.executor.memory", "512m") \
    .config("spark.executor.cores", "2") \
    .config("spark.cores.max", "4") \
    .config("spark.pyspark.python", worker_python) \
    .config("spark.pyspark.driver.python", driver_python) \
    .enableHiveSupport() \
    .getOrCreate()

print(f"✅ Sessão Segura Iniciada!")

# Teste Final
try:
    rdd = spark.sparkContext.parallelize(range(5), 5)
    worker = rdd.map(lambda x: socket.gethostname()).first()
    print(f"🚀 Processado por: {worker}")
except Exception as e:
    print(f"❌ Erro de Execução: {e}")

spark.stop()

🔌 Conectando em: spark://192.168.15.9:7077
📦 Storage: http://192.168.15.9:9000


26/01/04 12:16:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


✅ Sessão Segura Iniciada!


🚀 Processado por: localhost
